In [1]:
from UtilDatasetPrepFromLORE import *
from Rankers import *
from PreferentialSampling import *
from UniformSampling import *
from StandardEval import *
from FairnessEval import *
from Explainers import *

21-Dec-22 17:24:06 fatf.utils.array.tools INFO     Using numpy's numpy.lib.recfunctions.structured_to_unstructured as fatf.utils.array.tools.structured_to_unstructured and fatf.utils.array.tools.structured_to_unstructured_row.


In [2]:
Marit = ['Single','Single','Single','Married','Married','Divorced','Divorced','Divorced','Married','Married']
Age = ['Y','Y','Y','Y','Y','A','A','A','A','A']
House = ['Rent','Rent','Own','Own','Rent','Rent','Own','Own','Rent','Rent']
Cred = ['No','Bad','No','Excll','Excll','Bad','Excll','No','Bad','No']
Cl = [0,0,1,1,0,1,1,1,0,1]
d = {'Marit': Marit, 'Age': Age,
    'House': House, 'Cred': Cred,
    'Cl': Cl}
df = pd.DataFrame.from_dict(d)
df

,Marit,Age,House,Cred,Cl
0,Single,Y,Rent,No,0
1,Single,Y,Rent,Bad,0
2,Single,Y,Own,No,1
3,Married,Y,Own,Excll,1
4,Married,Y,Rent,Excll,0
5,Divorced,A,Rent,Bad,1
6,Divorced,A,Own,Excll,1
7,Divorced,A,Own,No,1
8,Married,A,Rent,Bad,0
9,Married,A,Rent,No,1


In [3]:
sensitive = ['Age']
df = prepare_toy_dataset(df, sensitive)
df

,Marit,House,Cred,target,Age_A,Age_Y
0,2,1,2,0,0,1
1,2,1,0,0,0,1
2,2,0,2,1,0,1
3,1,0,1,1,0,1
4,1,1,1,0,0,1
5,0,1,0,1,1,0
6,0,0,1,1,1,0
7,0,0,2,1,1,0
8,1,1,0,0,1,0
9,1,1,2,1,1,0


In [7]:
target = 'target'
column_names = df.columns.values.tolist()
column_names.remove(target)
class_names=['-','+']
adClass=1
disClass=0
protected = ['Age_A','Age_Y']
protected_feature='ages' 
non_sensitive = [elem for elem in column_names if elem not in protected]
adAttr=None#'Age_A'
disAttr=None#'Age_Y'

# Uniform Sampling

In [8]:
findFreq(target,protected,df) 

({'Age_A': 1, 'Age_Y': 3}, {'Age_A': 4, 'Age_Y': 2})

In [9]:
df_new=UniformSampling(target,protected,df,adClass,disClass,adAttr,disAttr)

Counting labels = 0  {'Age_A': 1, 'Age_Y': 3}
Counting labels = 1  {'Age_A': 4, 'Age_Y': 2}
DN  ('Age_Y', 3)
FP  ('Age_A', 4)
DP  ('Age_Y', 2)
FN  ('Age_A', 1)
Len DN_df  3
Len FP_df  4
Len DP_df  2
Len FN_df  1
Len restData 0
DN_expected  2
1 to be removed
Len DN_df after sampling  2
FP_expected  3
1 to be removed
Len FP_df after sampling  3
DP_expected  3
1 to add
Len DP_df after sampling  3
FN_expected  2
1 to add
Len FN_df after sampling  2


elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [10]:
findFreq(target,protected,df_new)

({'Age_A': 2, 'Age_Y': 2}, {'Age_A': 3, 'Age_Y': 3})

In [11]:
df_new

,Marit,House,Cred,Age_A,Age_Y,target
7,2,0,2,0,1,1
5,2,0,2,0,1,1
2,0,0,1,1,0,1
9,1,1,0,1,0,0
6,1,0,1,0,1,1
3,0,0,2,1,0,1
1,1,1,1,0,1,0
4,1,1,2,1,0,1
8,1,1,0,1,0,0
0,2,1,0,0,1,0


In [12]:
df_new.duplicated()

7    False
5     True
2    False
9    False
6    False
3    False
1    False
4    False
8     True
0    False
dtype: bool

# Preferential Sampling

In [13]:
Y = df[target].values 
X = df.drop(target, 1)

In [14]:
lr=LogisticRegression()
param_grid = {'penalty' : ['l1', 'l2'], 'C' : np.logspace(-4, 4, 20), 'solver' : ['liblinear']}
clf = RandomizedSearchCV(lr, param_grid, random_state=0)
search = clf.fit(X, Y)
ranker = search.best_estimator_
print(ranker)

The least populated class in y has only 4 members, which is less than n_splits=5.


LogisticRegression(C=11.288378916846883, solver='liblinear')


In [15]:
df_new=PreferentialSampling(target,protected,ranker,df,adClass,disClass,adAttr,disAttr)

Counting labels = 0  {'Age_A': 1, 'Age_Y': 3}
Counting labels = 1  {'Age_A': 4, 'Age_Y': 2}
DN  ('Age_Y', 3)
FP  ('Age_A', 4)
DP  ('Age_Y', 2)
FN  ('Age_A', 1)
Len DN_df  3
Len FP_df  4
Len DP_df  2
Len FN_df  1
Len restData 0
DN_expected  2
1 to be removed
Len DN_df after sampling  2
FP_expected  3
1 to be removed
Len FP_df after sampling  3
DP_expected  3
1 to add
Len DP_df after sampling  3
FN_expected  2
1 to add
Len FN_df after sampling  2


elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [16]:
findFreq(target,protected,df_new)

({'Age_A': 2, 'Age_Y': 2}, {'Age_A': 3, 'Age_Y': 3})

In [17]:
df_new

,Marit,House,Cred,Age_A,Age_Y,target,Probas
3,0,0,1,1,0,1,0.995411
6,2,0,2,0,1,1,0.796425
0,2,1,2,0,1,0,0.192847
9,1,1,0,1,0,0,0.237554
5,1,0,1,0,1,1,0.810762
2,1,1,2,1,0,1,0.923648
4,0,0,2,1,0,1,0.999261
8,1,1,0,1,0,0,0.237554
7,2,0,2,0,1,1,0.796425
1,2,1,0,0,1,0,0.006116


In [18]:
df_new.duplicated()

3    False
6    False
0    False
9    False
5    False
2    False
4    False
8     True
7     True
1    False
dtype: bool

# Testing Discrimination Measure 

In [19]:
Sex = ['M','M','M','M','M','F','F','F','F','F']
Ethnicity = ['Native','Native','Native','Non-nat.','Non-nat.','Non-nat.','Native','Native','Non-nat.','Native']
HighestDegree = ['H.school','Univ','H.school','H.school','Univ','Univ','H.school','None','Univ','H.school']
JobType = ['Board','Board','Board','Healthcare','Healthcare','Education','Education','Healthcare','Education','Board']
Cl = [1,1,1,1,0,0,0,1,0,1]
d = {'Sex': Sex, 'Ethnicity': Ethnicity,
    'HighestDegree': HighestDegree, 'JobType': JobType,
    'Cl': Cl}
df = pd.DataFrame.from_dict(d)
df

,Sex,Ethnicity,HighestDegree,JobType,Cl
0,M,Native,H.school,Board,1
1,M,Native,Univ,Board,1
2,M,Native,H.school,Board,1
3,M,Non-nat.,H.school,Healthcare,1
4,M,Non-nat.,Univ,Healthcare,0
5,F,Non-nat.,Univ,Education,0
6,F,Native,H.school,Education,0
7,F,Native,None,Healthcare,1
8,F,Non-nat.,Univ,Education,0
9,F,Native,H.school,Board,1


In [20]:
sensitive = ['Sex']
df = prepare_toy_dataset(df, sensitive)
df

,Ethnicity,HighestDegree,JobType,target,Sex_F,Sex_M
0,0,0,0,1,0,1
1,0,2,0,1,0,1
2,0,0,0,1,0,1
3,1,0,2,1,0,1
4,1,2,2,0,0,1
5,1,2,1,0,1,0
6,0,0,1,0,1,0
7,0,1,2,1,1,0
8,1,2,1,0,1,0
9,0,0,0,1,1,0


In [21]:
target = 'target'
column_names = df.columns.values.tolist()
column_names.remove(target)
class_names=['-','+']
adClass=1
disClass=0
protected = ['Sex_M','Sex_F']
protected_feature='sexes' 
non_sensitive = [elem for elem in column_names if elem not in protected]
adAttr=None#'Sex_M'
disAttr=None#'Sex_F'

In [25]:
adAttr, disAttr = inputDiscrimination(df,target,protected,adClass,disClass,adAttr=None,disAttr=None)

In [26]:
Discrimination(df,target,adClass,adAttr,disAttr)

Favoured attribute value: [ Sex_M ] Freq.: [ 5 ] Labeled as + class: [ 4 ]
Unfavoured attribute value: [ Sex_F ] Freq.: [ 5 ] Labeled as + class: [ 2 ]


0.4